## Modelo CTGAN para Geração dos Dados Sintéticos
<hr>

<p align="justify">O modelo pré-treinado está disponível em: <a href="https://github.com/sdv-dev/CTGAN" target="_blank">https://github.com/sdv-dev/CTGAN</a>. Ele foi escolhido por se tratar de uma solução robusta e específica para a geração de dados tabulares sintéticos. O <strong>CTGAN</strong> (Conditional Tabular GAN) se destaca pela capacidade de lidar com distribuições complexas, dados desbalanceados e variáveis categóricas, tornando-se ideal para cenários onde é necessário preservar as características fidedignas dos dados reais, sem comprometer a privacidade ou a segurança das informações.</p>

### 0. Instalação da Biblioteca CTGAN

In [15]:
pip install ctgan

Note: you may need to restart the kernel to use updated packages.


### 1. Importação dos Dados Gerados para Treinamento

In [1]:
import pandas as pd

df = pd.read_csv("../dataset/training_generated_data/generated_data.csv")
df.head()

,Nome,Nascimento,CPF,RG,Endereço,Telefone,Login,Senha
0,Hadassa Pereira,1962-10-16,129.685.704-20,812560346,"Quadra Sá, 893, São José, 23314869 Lopes das F...",(67) 95961-1883,eldwgbxl187,CP0Gk6u7+#
1,José Pimenta,1959-06-19,701.235.498-41,028765138,"Jardim de Siqueira, 68, Caetano Furquim, 63647...",(40) 98416-3591,ijcrwtsi977,t$6iZySN_2
2,Dr. Lucas Fernandes,2009-05-23,946.513.287-09,531876421,"Chácara Mendonça, 39, Santa Inês, 80425-898 Cu...",(20) 97085-7985,zgwecwl908,YDVOg90g&@
3,Matteo da Rosa,1914-03-19,058.691.374-20,087465218,"Jardim de Cassiano, 72, Nova Floresta, 55686-0...",(52) 98108-7007,ovwa0758,dV$11Xa$6p
4,Luna Dias,2014-05-28,970.216.854-67,23457186X,"Favela Melo, 138, Nova America, 20442-459 Silv...",(11) 97183-8924,kejc3405,E4zR$lmd(x


### 2. Gerador de Dados Sintéticos com CTGAN

In [3]:
import pandas as pd
from ctgan import CTGAN
from sklearn.model_selection import train_test_split
import os

def load_and_prepare_data(filepath):
    """Carrega e prepara os dados para modelagem"""
    df = pd.read_csv(filepath)
    
    # Verificação dos dados
    print("\nAmostra dos dados de treinamento:")
    print(df.head())
    print(f"\nTotal de registros carregados: {len(df)}")
    
    # Pré-processamento para o CTGAN
    discrete_columns = ['Nome', 'CPF', 'RG', 'Login', 'Endereço', 'Telefone', 'Nascimento', 'Senha']
    for col in discrete_columns:
        df[col] = df[col].astype(str)
    
    return df, discrete_columns

def train_ctgan(train_data, discrete_columns):
    """Instância o Modelo CTGAN"""
    ctgan = CTGAN(
        epochs=15,  
        batch_size=200,
        verbose=True
    )
    
    print("\nIniciando treinamento do modelo CTGAN...")
    ctgan.fit(train_data, discrete_columns)
    return ctgan

def generate_synthetic_data(ctgan_model):
    """Gera dados sintéticos com tamanho especificado pelo usuário"""
    while True:
        try:
            num_samples = int(input("\nQuantos registros sintéticos deseja gerar? (Digite um número inteiro): "))
            if num_samples > 0:
                break
            else:
                print("Por favor, digite um número positivo.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número inteiro.")
    
    print(f"\nGerando {num_samples} registros sintéticos...")
    return ctgan_model.sample(num_samples)

def ensure_directory_exists(path):
    """Garante que o diretório existe, criando se necessário"""
    os.makedirs(path, exist_ok=True)

def main():
    # Configurações de caminhos
    input_file = '../dataset/training_generated_data/generated_data.csv'
    synthetic_data_dir = '../synthetic_data/'  # Pasta para dados sintéticos
    training_data_dir = '../training_data/'    # Pasta para dados de treino/teste
    
    # Nomes dos arquivos de saída
    synthetic_file = 'synthetic_secure_data.csv'
    train_data_file = 'train_data.csv'
    test_data_file = 'test_data.csv'
    
    # Garante que os diretórios existam
    ensure_directory_exists(synthetic_data_dir)
    ensure_directory_exists(training_data_dir)
    
    # Carrega e prepara os dados
    df, discrete_columns = load_and_prepare_data(input_file)
    
    # Split dos dados em treino e teste (80/20)
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    print(f"\nDados divididos - Treino: {len(train_data)} registros | Teste: {len(test_data)} registros")
    
    # Salva os dados de treino e teste
    train_data_path = os.path.join(training_data_dir, train_data_file)
    test_data_path = os.path.join(training_data_dir, test_data_file)
    
    train_data.to_csv(train_data_path, index=False)
    test_data.to_csv(test_data_path, index=False)
    print(f"Dados de treino salvos em: {train_data_path}")
    print(f"Dados de teste salvos em: {test_data_path}")
    
    # Treina o modelo
    ctgan_model = train_ctgan(train_data, discrete_columns)
    
    # Gera dados sintéticos
    synthetic_data = generate_synthetic_data(ctgan_model)
    
    # Mostra amostra dos dados gerados
    print("\nAmostra de dados sintéticos gerados:")
    print(synthetic_data.head())
    
    # Salva os dados sintéticos
    synthetic_path = os.path.join(synthetic_data_dir, synthetic_file)
    synthetic_data.to_csv(synthetic_path, index=False)
    print(f"\nDados sintéticos salvos em: {synthetic_path}")

if __name__ == "__main__":
    main()


Amostra dos dados de treinamento:
                  Nome  Nascimento             CPF         RG  \
0      Hadassa Pereira  1962-10-16  129.685.704-20  812560346   
1         José Pimenta  1959-06-19  701.235.498-41  028765138   
2  Dr. Lucas Fernandes  2009-05-23  946.513.287-09  531876421   
3       Matteo da Rosa  1914-03-19  058.691.374-20  087465218   
4            Luna Dias  2014-05-28  970.216.854-67  23457186X   

                                            Endereço         Telefone  \
0  Quadra Sá, 893, São José, 23314869 Lopes das F...  (67) 95961-1883   
1  Jardim de Siqueira, 68, Caetano Furquim, 63647...  (40) 98416-3591   
2  Chácara Mendonça, 39, Santa Inês, 80425-898 Cu...  (20) 97085-7985   
3  Jardim de Cassiano, 72, Nova Floresta, 55686-0...  (52) 98108-7007   
4  Favela Melo, 138, Nova America, 20442-459 Silv...  (11) 97183-8924   

         Login       Senha  
0  eldwgbxl187  CP0Gk6u7+#  
1  ijcrwtsi977  t$6iZySN_2  
2   zgwecwl908  YDVOg90g&@  
3     ovwa0758  dV$

Gen. (6.73) | Discrim. (0.01): 100%|███████████████████████████████████████████████████| 15/15 [01:44<00:00,  6.99s/it]



Quantos registros sintéticos deseja gerar? (Digite um número inteiro):  100



Gerando 100 registros sintéticos...

Amostra de dados sintéticos gerados:
               Nome  Nascimento             CPF         RG  \
0     Levi Mendonça  1931-06-16  671.905.328-03  761805230   
1   Isabelly Duarte  1976-04-12  623.089.417-31  106842572   
2   Sr. Juan Aragão  1955-08-25  346.107.295-80  037528610   
3     Marcelo Abreu  1949-11-19  840.319.527-32  452816300   
4  Lívia Cavalcante  1974-01-21  018.697.435-39  165732088   

                                            Endereço         Telefone  \
0  Largo de Novaes, 512, Cônego Pinheiro 2ª Seção...  (27) 96762-5042   
1  Viela Sampaio, 883, Vila Da Paz, 06942-243 Dia...  (06) 97203-6818   
2  Quadra Matheus Montenegro, 642, Marmiteiros, 6...  (96) 90069-7479   
3  Conjunto de Cardoso, 5, Calafate, 65042019 Rio...  (62) 91419-6876   
4  Largo de Carvalho, 926, São Vicente, 44219-475...  (02) 90856-9021   

          Login       Senha  
0   zfpyptcj886  fN#a8Oenc#  
1  embjxytp0857  fa5TLP#N%$  
2     hgnlr8192  qdDBkg

In [5]:
import pandas as pd
from ctgan import CTGAN
from sklearn.model_selection import train_test_split
import os

def load_and_prepare_data(filepath):
    """Carrega e prepara os dados para modelagem"""
    df = pd.read_csv(filepath)
    
    # Verificação dos dados
    print("\nAmostra dos dados de treinamento:")
    print(df.head())
    print(f"\nTotal de registros carregados: {len(df)}")
    
    # Pré-processamento para o CTGAN
    discrete_columns = ['Nome', 'CPF', 'RG', 'Login', 'Endereço', 'Telefone', 'Nascimento', 'Senha']
    for col in discrete_columns:
        df[col] = df[col].astype(str)
    
    return df, discrete_columns

def train_ctgan(train_data, discrete_columns, epochs=15, batch_size=200):
    """Instância o Modelo CTGAN e treina"""
    ctgan = CTGAN(epochs=epochs, batch_size=batch_size, verbose=True)
    print("\nIniciando treinamento do modelo CTGAN...")
    ctgan.fit(train_data, discrete_columns)
    return ctgan

def generate_synthetic_data(ctgan_model, num_samples):
    """Gera dados sintéticos com o número de amostras especificado"""
    print(f"\nGerando {num_samples} registros sintéticos...")
    return ctgan_model.sample(num_samples)

def ensure_directory_exists(path):
    """Garante que o diretório exista, criando se necessário"""
    os.makedirs(path, exist_ok=True)

def split_and_save_data(df, train_data_dir, test_data_dir, train_data_file, test_data_file):
    """Divide os dados em treino e teste e salva os arquivos"""
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    print(f"\nDados divididos - Treino: {len(train_data)} registros | Teste: {len(test_data)} registros")
    
    # Salva os dados de treino e teste
    train_data_path = os.path.join(train_data_dir, train_data_file)
    test_data_path = os.path.join(test_data_dir, test_data_file)
    
    train_data.to_csv(train_data_path, index=False)
    test_data.to_csv(test_data_path, index=False)
    print(f"Dados de treino salvos em: {train_data_path}")
    print(f"Dados de teste salvos em: {test_data_path}")
    
    return train_data

def main():
    # Configurações de caminhos
    input_file = '../dataset/training_generated_data/generated_data.csv'
    synthetic_data_dir = '../synthetic_data/'  # Pasta para dados sintéticos
    training_data_dir = '../training_data/'    # Pasta para dados de treino/teste
    
    # Nomes dos arquivos de saída
    synthetic_file = 'synthetic_secure_data.csv'
    train_data_file = 'train_data.csv'
    test_data_file = 'test_data.csv'
    
    # Garante que os diretórios existam
    ensure_directory_exists(synthetic_data_dir)
    ensure_directory_exists(training_data_dir)
    
    # Carrega e prepara os dados
    df, discrete_columns = load_and_prepare_data(input_file)
    
    # Divide e salva os dados em treino e teste
    train_data = split_and_save_data(df, training_data_dir, training_data_dir, train_data_file, test_data_file)
    
    # Treina o modelo
    ctgan_model = train_ctgan(train_data, discrete_columns)
    
    # Solicita ao usuário a quantidade de dados sintéticos a serem gerados
    while True:
        try:
            num_samples = int(input("\nQuantos registros sintéticos deseja gerar? (Digite um número inteiro): "))
            if num_samples > 0:
                break
            else:
                print("Por favor, digite um número positivo.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número inteiro.")
    
    # Gera os dados sintéticos
    synthetic_data = generate_synthetic_data(ctgan_model, num_samples)
    
    # Mostra amostra dos dados gerados
    print("\nAmostra de dados sintéticos gerados:")
    print(synthetic_data.head())
    
    # Salva os dados sintéticos
    synthetic_path = os.path.join(synthetic_data_dir, synthetic_file)
    synthetic_data.to_csv(synthetic_path, index=False)
    print(f"\nDados sintéticos salvos em: {synthetic_path}")

if __name__ == "__main__":
    main()



Amostra dos dados de treinamento:
                  Nome  Nascimento             CPF         RG  \
0      Hadassa Pereira  1962-10-16  129.685.704-20  812560346   
1         José Pimenta  1959-06-19  701.235.498-41  028765138   
2  Dr. Lucas Fernandes  2009-05-23  946.513.287-09  531876421   
3       Matteo da Rosa  1914-03-19  058.691.374-20  087465218   
4            Luna Dias  2014-05-28  970.216.854-67  23457186X   

                                            Endereço         Telefone  \
0  Quadra Sá, 893, São José, 23314869 Lopes das F...  (67) 95961-1883   
1  Jardim de Siqueira, 68, Caetano Furquim, 63647...  (40) 98416-3591   
2  Chácara Mendonça, 39, Santa Inês, 80425-898 Cu...  (20) 97085-7985   
3  Jardim de Cassiano, 72, Nova Floresta, 55686-0...  (52) 98108-7007   
4  Favela Melo, 138, Nova America, 20442-459 Silv...  (11) 97183-8924   

         Login       Senha  
0  eldwgbxl187  CP0Gk6u7+#  
1  ijcrwtsi977  t$6iZySN_2  
2   zgwecwl908  YDVOg90g&@  
3     ovwa0758  dV$

Gen. (6.79) | Discrim. (-0.01): 100%|██████████████████████████████████████████████████| 15/15 [01:41<00:00,  6.74s/it]



Quantos registros sintéticos deseja gerar? (Digite um número inteiro):  1000



Gerando 1000 registros sintéticos...

Amostra de dados sintéticos gerados:
                    Nome  Nascimento             CPF         RG  \
0         Larissa Vargas  1956-06-08  215.604.739-16  543021671   
1  Dr. José Miguel Sales  2024-05-31  203.497.815-32  436805716   
2  Sra. Marcela Caldeira  1991-11-09  547.109.628-20  834627152   
3         Maria Mendonça  2017-06-19  963.084.752-38  153480725   
4        Camila Monteiro  1934-06-27  436.987.021-69  507326489   

                                            Endereço         Telefone  \
0  Pátio de Nogueira, 14, Prado, 84883-575 Mendes...  (05) 96292-6307   
1  Travessa Almeida, 19, Santa Terezinha, 3833248...  (02) 90856-9021   
2  Largo de Sampaio, 5, Tirol, 31125311 Aparecida...  (51) 99259-5495   
3  Estação Carolina Ferreira, 9, Jardim Atlântico...  (06) 97203-6818   
4  Sítio de Viana, 63, Vila Da Paz, 10841-688 Car...  (33) 98750-5846   

         Login       Senha  
0   isvgjc5926  +h0TKtBuao  
1      bice897  F2EVrVm@